## RESULTS:
**Close-ended Questions:**

Final-Answer Accuracy: 79.39%  --> 208/262 were answered correct

Rationale BERTScore-F1: 85.57%

-----------------------------------------------------


**Open-ended Questions:**

Final-Answer BERTScore-F1: 87.82%

Rationale BERTScore-F1: 84.81%



In [3]:
import pandas as pd

In [4]:
close = pd.read_csv("/content/Deepseek KG+RAG - CLOSED.csv")

In [5]:
close.head()

,question,choices,answer,image,question_type,answer_type,image_organ,phrase_type,solution,id,split,image_processed,generated_response
0,Is there evidence of an aortic aneurysm?,"['yes', 'no']",yes,synpic42202.jpg,PRES,CLOSED,CHEST,freeform,The presence of a widened mediastinum on the X...,11,test,<PIL.Image.Image image mode=RGB size=512x512 a...,**Final Answer:** \nno \n\n**Rationale:** \...
1,Is there airspace consolidation on the left side?,"['yes', 'no']",yes,synpic29265.jpg,PRES,CLOSED,CHEST,freeform,"On the provided chest X-ray, there is evidence...",13,test,<PIL.Image.Image image mode=RGB size=512x512 a...,**Final Answer:** \nno \n\n**Rationale:** \...
2,Is this an axial plane?,"['yes', 'no']",yes,synpic54610.jpg,PLANE,CLOSED,HEAD,freeform,"An axial plane, also known as a transverse pla...",14,test,<PIL.Image.Image image mode=RGB size=512x512 a...,\n**Final Answer:** \nyes \n\n**Rationale:**...
3,Is there vascular pathology in this image?,"['yes', 'no']",yes,synpic54610.jpg,ABN,CLOSED,HEAD,freeform,The image provided is a diffusion-weighted MRI...,22,test,<PIL.Image.Image image mode=RGB size=512x512 a...,**Final Answer:** \nyes \n\n**Rationale:** ...
4,Is there cardiomyopathy?,"['yes', 'no']",yes,synpic42202.jpg,SIZE,CLOSED,CHEST,freeform,The image provided is a chest X-ray showing an...,24,test,<PIL.Image.Image image mode=RGB size=512x512 a...,**Final Answer:** \nno \n\n**Rationale:** \...


In [6]:
close["generated_response"][4]

'**Final Answer:**  \nno  \n\n**Rationale:**  \nThe provided context does not contain any direct evidence or discussion of cardiomyopathy. Cardiomyopathy involves structural or functional abnormalities of the heart muscle, which might manifest on imaging as cardiomegaly, chamber enlargement, or other specific features. Among the retrieved cases:  \n- Case 3 references the cardiovascular system due to a pacemaker, but this relates to electrical conduction issues (e.g., arrhythmias) rather than structural heart muscle disease.  \n- Other cases focus on pneumothorax, image orientation, and abdominal findings, none of which address cardiac size, shape, or function.  \nSince no context explicitly supports signs of cardiomyopathy (e.g., heart enlargement, chamber dilation, or clinical indicators like heart failure), the answer is "no" based on the absence of relevant evidence in the provided cases.'

In [7]:
!pip install pandas bert-score torch tqdm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:00


In [10]:
import pandas as pd
import re
from bert_score import score

# 1) Load your data and drop NaN rows
df = pd.read_csv("/content/Deepseek KG+RAG - CLOSED.csv")
df = df.dropna()  # Drops rows with any NaN values

# 2) Helpers to split out answer vs. rationale
def extract_final_answer(text):
    # Handle non-string inputs (e.g., float, NaN)
    if not isinstance(text, str) or pd.isna(text):
        return ""
    # Match "Final Answer:" or variations, capturing everything after it
    m = re.search(r'\*\*Final Answer:\*\*\s*(.*?)(?:\n\n|$)', text, re.IGNORECASE | re.DOTALL)
    return m.group(1).strip() if m else text.strip()

def extract_rationale(text):
    # Handle non-string inputs (e.g., float, NaN)
    if not isinstance(text, str) or pd.isna(text):
        return ""
    # Capture everything after "**Rationale:**"
    m = re.search(r'\*\*Rationale:\*\*\s*(.*)', text, re.IGNORECASE | re.DOTALL)
    return m.group(1).strip() if m else ""

# Apply the functions
df['predicted_answer'] = df['generated_response'].apply(extract_final_answer)
df['rationale'] = df['generated_response'].apply(extract_rationale)

# 3) Drop rows where rationale or solution is empty
df = df[df['rationale'].str.strip() != ""]  # Drop rows with empty rationale
df = df[df['solution'].str.strip() != ""]   # Drop rows with empty solution

# 4) Number of correct final answers and accuracy
df['correct'] = (
    df['predicted_answer'].str.lower().str.strip()
    == df['answer'].str.lower().str.strip()
)
num_correct = df['correct'].sum()
overall_acc = df['correct'].mean()
print(f"Number of Correct Final Answers: {num_correct}")
print(f"Overall Final-Answer Accuracy: {overall_acc:.2%}\n")

# 5) BERTScore between rationale vs. solution
P_rat, R_rat, F1_rat = score(
    df['rationale'].tolist(),
    df['solution'].tolist(),
    lang="en", verbose=True
)
print(f" Rationale BERTScore →  P={P_rat.mean():.4f}, R={R_rat.mean():.4f}, F1={F1_rat.mean():.4f}\n")

# 6) Breakdown by answer type
grouped = (
    df.groupby('answer_type')['correct']
      .agg(correct_count='sum', total_count='count')
      .reset_index()
)
grouped['accuracy'] = grouped['correct_count'] / grouped['total_count']
print("Breakdown by Answer Type:")
print(grouped.to_string(index=False))

Number of Correct Final Answers: 208
Overall Final-Answer Accuracy: 79.39%



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 12.27 seconds, 21.35 sentences/sec
 Rationale BERTScore →  P=0.8455, R=0.8664, F1=0.8558

Breakdown by Answer Type:
answer_type  correct_count  total_count  accuracy
     CLOSED            208          262  0.793893


In [ ]:
openq = pd.read_csv("/content/Deepseek KG+RAG - OPEN.csv")

In [ ]:
openq.head()

,question,choices,answer,image,question_type,answer_type,image_organ,phrase_type,solution,id,split,image_processed,generated_response
0,How is the patient oriented?,['Posterior-Anterior'],Posterior-Anterior,synpic29265.jpg,OTHER,OPEN,CHEST,freeform,The orientation of the patient for this X-ray ...,20,test,<PIL.Image.Image image mode=RGB size=512x512 a...,\n\n**Final Answer:** \nThe patient is orient...
1,Describe the lung abnormalities?,['Pulmonary nodules'],Pulmonary nodules,synpic18319.jpg,ABN,OPEN,CHEST,para,This chest x-ray demonstrates several rounded ...,302,test,<PIL.Image.Image image mode=RGB size=512x512 a...,\n\n**Final Answer:** \nPulmonary nodules \n...
2,What abnormalities are seen within the lungs?,['Pulmonary nodules'],Pulmonary nodules,synpic18319.jpg,ABN,OPEN,CHEST,freeform,Pulmonary nodules can be identified on a chest...,304,test,<PIL.Image.Image image mode=RGB size=512x512 a...,\n\n**Final Answer:** \nAreas of increased op...
3,What is under the right hemidiaphragm?,['Free air'],Free air,synpic32933.jpg,PRES,OPEN,CHEST,freeform,The presence of free air under the right hemid...,306,test,<PIL.Image.Image image mode=RGB size=512x512 a...,\n\n**Final Answer:** \nLiver \n\n**Rational...
4,What is the abnormality by the right hemidiaph...,['Free air'],Free air,synpic32933.jpg,PRES,OPEN,CHEST,para,The abnormality indicated by the free air sugg...,308,test,<PIL.Image.Image image mode=RGB size=512x512 a...,NaN


In [ ]:
openq["generated_response"][9]

'\n\n**Final Answer:**  \nfatty infiltration  \n\n**Rationale:**  \nThe user\'s query asks about abnormalities in the pancreas. Among the retrieved context cases:  \n- **Cases 1 and 2** describe pancreatic edema and enlargement, but these answers are tied to specific questions about edema or organ enlargement, not a general inquiry about abnormalities.  \n- **Cases 4 and 5** directly address the question "What is wrong with the pancreas?" and both conclude **fatty infiltration** as the abnormality. The rationales highlight CT findings such as low-density areas, heterogeneous texture, and attenuation values resembling subcutaneous fat, which align with fatty infiltration.  \n- **Case 3** identifies the pancreas as the location of an abnormality but does not specify the pathology.  \n\nSince the user’s question closely mirrors the structure of Cases 4 and 5 (general inquiry about pancreatic abnormalities) and these cases provide a consistent answer supported by imaging characteristics, *

In [11]:
import pandas as pd
import re
from bert_score import score

df = pd.read_csv("/content/Deepseek KG+RAG - OPEN.csv")

def extract_final_answer(text):
    if not isinstance(text, str) or pd.isna(text):
        return ""
    m = re.search(r'\*\*Final Answer:\*\*\s*(.*?)(?:\n\n|$)', text, re.IGNORECASE | re.DOTALL)
    return m.group(1).strip() if m else text.strip()

def extract_rationale(text):
    if not isinstance(text, str) or pd.isna(text):
        return ""
    m = re.search(r'\*\*Rationale:\*\*\s*(.*)', text, re.IGNORECASE | re.DOTALL)
    return m.group(1).strip() if m else ""

df['predicted_answer'] = df['generated_response'].apply(extract_final_answer)
df['rationale'] = df['generated_response'].apply(extract_rationale)

df = df[(df['predicted_answer'].str.strip() != "") & (df['rationale'].str.strip() != "")]
df = df.dropna(subset=['answer', 'solution'])
print(f"Number of rows after dropping empty entries: {len(df)}")

print("\nSample of Extracted Data:")
sample = df[['generated_response', 'predicted_answer', 'rationale']].iloc[0]
print("\nOriginal generated_response:")
print(sample['generated_response'])
print("\nExtracted predicted_answer:")
print(sample['predicted_answer'])
print("\nExtracted rationale:")
print(sample['rationale'])
print("\n" + "-"*50 + "\n")

P_ans, R_ans, F1_ans = score(
    df['predicted_answer'].tolist(),
    df['answer'].tolist(),
    lang="en", verbose=True
)

P_rat, R_rat, F1_rat = score(
    df['rationale'].tolist(),
    df['solution'].tolist(),
    lang="en", verbose=True
)

print(f"Final Answer BERTScore →  P={P_ans.mean():.4f}, R={R_ans.mean():.4f}, F1={F1_ans.mean():.4f}")
print(f"Rationale    BERTScore →  P={P_rat.mean():.4f}, R={R_rat.mean():.4f}, F1={F1_rat.mean():.4f}")

Number of rows after dropping empty entries: 160

Sample of Extracted Data:

Original generated_response:


**Final Answer:**  
The patient is oriented with the left side of the image corresponding to the patient's right side (standard radiographic convention).  

**Rationale:**  
In Context Case 1, the rationale explicitly states that a density seen on the left side of the image corresponds to the patient's right lung. This aligns with the standard orientation of chest X-rays, where the image is displayed as if the patient is facing the viewer (anteroposterior or posteroanterior projection), resulting in the left side of the image representing the patient's anatomical right. None of the other context cases contradict this orientation. The remaining cases focus on other findings (e.g., pneumothorax, MRI vs. X-ray, fractures, air under the diaphragm) but do not address orientation, leaving Case 1 as the sole and sufficient evidence for determining patient orientation.

Extracted predict

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


...

calculating scores...
computing bert embedding.


  0%|          | 0/5 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 7.77 seconds, 20.58 sentences/sec
Final Answer BERTScore →  P=0.8763, R=0.8809, F1=0.8782
Rationale    BERTScore →  P=0.8412, R=0.8552, F1=0.8481
